In [1]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import re
import itertools
from datetime import date
import datetime
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

We have two files:
1- Diabetes patients admission encounter diabetes_data.csv.
2- Mapping file that can be used to make the variables have clear meaning.

In [2]:
# reading data
f=open('../data/raw/IDs_mapping.csv','r')
ids_mappings=f.read()
diabetes_data=pd.read_csv('../data/raw/diabetes_data.csv')

In [3]:
diabetes_data.head()

,encounter_id,patient_nbr,race,gender,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admit_date,birth_date
0,2278392,8222157,Caucasian,Female,?,6,25,1,1,?,...,No,No,No,No,No,No,No,NO,2002-10-22 06:30:00,1996-07-29 14:28:00
1,149190,55629189,Caucasian,Female,?,1,1,7,3,?,...,No,No,No,No,No,Ch,Yes,>30,2004-07-24 08:35:00,1985-09-19 11:19:00
2,64410,86047875,AfricanAmerican,Female,?,1,1,7,2,?,...,No,No,No,No,No,No,Yes,NO,2005-11-30 06:35:00,1983-07-14 07:22:00
3,500364,82442376,Caucasian,Male,?,1,1,7,2,?,...,No,No,No,No,No,Ch,Yes,NO,2003-05-26 00:32:00,1968-04-05 23:51:00
4,16680,42519267,Caucasian,Male,?,1,1,7,1,?,...,No,No,No,No,No,Ch,Yes,NO,2004-04-25 04:04:00,1957-11-18 20:30:00


The mapping dataset is not typical. It has more than one table in once csv file separated by a commas. To use them for mapping the features later, I convert each table to a dictionary.
the logic:
take the mapping table "ids_mappings" text then extract the tables first. Then extract the rows and split them into rows. The individual elements will serve as either key or value in the final dictionary. 

In [4]:
'''
 IDs_mapping.csv has mutiple tables in one file which is not a typical use of CSV. So I decided to read it as text then creat lists to map the numeric value 
 in the feature dataframe to more understandable values.
'''
def convert_list_to_dict(lst):
    pairs=itertools.zip_longest(*[iter(lst)] * 2, fillvalue=None)
    dct={key: value for key, value in pairs}
    return dct

# split text to get tables
codes_mapping_tables=ids_mappings.lower().split('\n,\n')

# Extract tables' rows
mapping_tables_dict={}
for mapping_table in codes_mapping_tables:
    #print(mapping_table)
    # Extract columns from rows
    rows = re.split('\n',mapping_table)
    clean_cols=[]
    for row in rows:
        cols=row.split(',')
        if len(cols)>2:
            #print(cols)
            for i in range(2,len(cols)):
                cols[1]+=cols[i]
            cols=cols[:2]
        clean_cols += cols
    #print(clean_cols)
    clean_mapping_table=[x.replace('/','_').replace(' ','_').replace('.','').replace('"','') for x in clean_cols]
    #print(clean_mapping_table)
    mapping_tables_dict[clean_cols[0]]=convert_list_to_dict(clean_mapping_table)

# Clean up the dictionary
del mapping_tables_dict['admission_source_id']['']
mapping_tables_dict['admission_type_id']['6']='admission_type_6'
mapping_tables_dict['discharge_disposition_id']['18']='discharge_disposition_18'
mapping_tables_dict

{'admission_type_id': {'admission_type_id': 'description',
  '1': 'emergency',
  '2': 'urgent',
  '3': 'elective',
  '4': 'newborn',
  '5': 'not_available',
  '6': 'admission_type_6',
  '7': 'trauma_center',
  '8': 'not_mapped'},
 'discharge_disposition_id': {'discharge_disposition_id': 'description',
  '1': 'discharged_to_home',
  '2': 'discharged_transferred_to_another_short_term_hospital',
  '3': 'discharged_transferred_to_snf',
  '4': 'discharged_transferred_to_icf',
  '5': 'discharged_transferred_to_another_type_of_inpatient_care_institution',
  '6': 'discharged_transferred_to_home_with_home_health_service',
  '7': 'left_ama',
  '8': 'discharged_transferred_to_home_under_care_of_home_iv_provider',
  '9': 'admitted_as_an_inpatient_to_this_hospital',
  '10': 'neonate_discharged_to_another_hospital_for_neonatal_aftercare',
  '11': 'expired',
  '12': 'still_patient_or_expected_to_return_for_outpatient_services',
  '13': 'hospice___home',
  '14': 'hospice___medical_facility',
  '15': '

2- After reading the dataset diabetes_data, we start exploring the following:
- Variables that make sense to use.
In the table below, discharge_disposition_id column will not be used because the model is going to be used in production before discharge. 
- Birth_date & admit_date can be used to create age during this inpatient encounter (age = admit_date - birth_date )
- Encounter_id & patient_nbr are going to be used for data preparation but not as features.  


In [5]:
# Display the dataframe information 
diabetes_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   weight                    101766 non-null  object
 5   admission_type_id         101766 non-null  int64 
 6   discharge_disposition_id  101766 non-null  int64 
 7   admission_source_id       101766 non-null  int64 
 8   time_in_hospital          101766 non-null  int64 
 9   payer_code                101766 non-null  object
 10  medical_specialty         101766 non-null  object
 11  num_lab_procedures        101766 non-null  int64 
 12  num_procedures            101766 non-null  int64 
 13  num_medications           101766 non-null  int64 
 14  numb

In [6]:
# Removing discharge_disposition_id from the data since it is not going to be used
diabetes_data.drop(axis=1, labels=['discharge_disposition_id'], inplace=True)

In [7]:
# Calculate age by first converting the columns from object to datetime64 
diabetes_data['admit_date'] = pd.to_datetime(diabetes_data['admit_date'])
diabetes_data['birth_date'] = pd.to_datetime(diabetes_data['birth_date'])
diabetes_data['admit_date'] = diabetes_data['admit_date'].map(datetime.datetime.date)
diabetes_data['birth_date'] = diabetes_data['birth_date'].map(datetime.datetime.date)
diabetes_data['age'] = ((diabetes_data['admit_date']- diabetes_data['birth_date']).apply(lambda x: x.days)/365.2425)


3- Check the categorical variables values counts. 
- Race will be converted to 3 values Caucasian, AfricanAmerican, and other. The column is very biased to spesific group so if we are going to use the model in production, this should be avoided.
- The "Weight" columns is mostly unknow so it is safe to drop it for this dataset.
- Payer_code has 39.5% missing values. The payer will have a single to socioeconomic status which might predict readmission. I will treat the missing values as other.
- Medical_specialty has 49% missing values. I will treat them as other. 

In [8]:
# Remove columns with 80% missing values
diabetes_data.replace('?',np.nan,inplace=True)
diabetes_data = diabetes_data.loc[:, diabetes_data.isnull().mean() <.8]

In [9]:
diabetes_data.isnull().sum()/len(diabetes_data)*100

encounter_id                 0.000000
patient_nbr                  0.000000
race                         2.233555
gender                       0.000000
admission_type_id            0.000000
admission_source_id          0.000000
time_in_hospital             0.000000
payer_code                  39.557416
medical_specialty           49.082208
num_lab_procedures           0.000000
num_procedures               0.000000
num_medications              0.000000
number_outpatient            0.000000
number_emergency             0.000000
diag_1                       0.020636
diag_2                       0.351787
diag_3                       1.398306
number_diagnoses             0.000000
metformin                    0.000000
repaglinide                  0.000000
nateglinide                  0.000000
chlorpropamide               0.000000
glimepiride                  0.000000
acetohexamide                0.000000
glipizide                    0.000000
glyburide                    0.000000
tolbutamide 

In [10]:
diabetes_data['payer_code'].fillna('other',inplace=True)
diabetes_data['medical_specialty'].fillna('other',inplace=True)
diabetes_data['diag_1'].fillna('no_diag',inplace=True)
diabetes_data['diag_2'].fillna('no_diag',inplace=True)
diabetes_data['diag_3'].fillna('no_diag',inplace=True)

4- Create the label 

In [11]:
diabetes_data['label'] = diabetes_data['readmitted'].apply(lambda x: 1 if x=='<30' else 0)
diabetes_data['label'].value_counts()

label
0    90409
1    11357
Name: count, dtype: int64

5- Invistigating having mutiple rows per patient and its possible impact on training the model. It can be used to impute columns such as age, gender, and race. There are around 800 members with inpatient admission rows >5. I will leave them for now but they might become a source of noise. 

In [12]:
# Counts 
print('Unique patients: ', len(diabetes_data['patient_nbr'].unique()), ' Dataset rows: ',len(diabetes_data))

Unique patients:  71518  Dataset rows:  101766


In [13]:
# Grouping by patient_nbr to check the how many row per patient and the label distribution
patients_groups = diabetes_data.groupby(by='patient_nbr')
# most people has 
print('maximum number of admissions for one patient in the dataset: ',max([len(g[1]) for g in patients_groups]))

colms =['patient_nbr','race','payer_code','admit_date','birth_date']
rows_per_patient = {}
for g in patients_groups:
    if len(g[1])>1:
        rows_per_patient[g[1]['patient_nbr'].unique()[0]]= g[1]['label'].values

rows_per_patient
   

maximum number of admissions for one patient in the dataset:  40


{135: array([1, 0]),
 1152: array([0, 0, 0, 0, 0]),
 1314: array([0, 1, 0]),
 5220: array([0, 0, 0, 0]),
 5337: array([0, 0]),
 6696: array([0, 0]),
 10827: array([0, 0]),
 11394: array([0, 1, 0]),
 11511: array([1, 1, 0, 0]),
 12447: array([0, 0, 0, 0]),
 13041: array([0, 1, 1, 0, 0, 0, 0, 0]),
 16425: array([0, 0]),
 18765: array([0, 0, 0, 0]),
 20232: array([1, 0]),
 20295: array([0, 0]),
 20997: array([0, 0]),
 24030: array([0, 0]),
 24525: array([0, 0, 0]),
 25362: array([0, 0]),
 27315: array([0, 0]),
 29961: array([0, 0, 0]),
 30735: array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0]),
 31320: array([1, 0, 0, 0]),
 32337: array([0, 0]),
 32733: array([0, 0]),
 32967: array([1, 0]),
 33264: array([0, 0]),
 33354: array([0, 0, 1, 1, 1, 0, 0]),
 34353: array([0, 0, 0]),
 35271: array([0, 0, 0]),
 36810: array([1, 0, 0]),
 36999: array([1, 0]),
 37125: array([0, 0]),
 37305: array([0, 0]),
 37863: array([0, 0, 0]),
 39069: array([0, 0]),
 40158: array([0, 0, 0, 0, 0, 0]),
 42120: array([0, 0, 0])

6- Displaying numerical columns summary. Looking at age the minimum values looks strange and if it is for an infant, a question for the stakeholder would be about the age restrictions for the model. 

In [14]:
diabetes_data.describe()

,encounter_id,patient_nbr,admission_type_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_diagnoses,age,label
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,7.422607,66.544648,0.111599
std,1.026403e+08,3.869636e+07,1.445403,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.933600,16.295346,0.314874
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.150585,0.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,6.000000,55.996359,0.000000
50%,1.523890e+08,4.550514e+07,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,8.000000,68.453151,0.000000
75%,2.302709e+08,8.754595e+07,3.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,9.000000,78.493056,0.000000
max,4.438672e+08,1.895026e+08,8.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,16.000000,107.616173,1.000000


7- Convert admission_type_id and admission_source_id into categorical variables and use english instead of number to describe them

In [15]:
diabetes_data['admission_type_id'] = diabetes_data['admission_type_id'].apply(lambda x: str(x)).map(mapping_tables_dict['admission_type_id'])
diabetes_data['admission_source_id'] = diabetes_data['admission_source_id'].apply(lambda x: str(x)).map(mapping_tables_dict['admission_source_id'])

8- Drop the columns that we don't want anymore, create summy variables, split the data, and save the processed files.

In [16]:
diabetes_data.drop(axis=1 , labels=['readmitted', 'admit_date', 'birth_date','patient_nbr','encounter_id'], inplace=True)

In [17]:
'''
categorical_columns = ['race', 'gender', 'admission_type_id',
       'admission_source_id', 'time_in_hospital', 'payer_code',
       'medical_specialty', 'diag_1',
       'diag_2', 'diag_3', 'metformin', 'repaglinide',
       'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
       'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
       'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
       'glipizide-metformin', 'glimepiride-pioglitazone',
       'metformin-rosiglitazone', 'metformin-pioglitazone','change','diabetesMed'
       ]
'''

"\ncategorical_columns = ['race', 'gender', 'admission_type_id',\n       'admission_source_id', 'time_in_hospital', 'payer_code',\n       'medical_specialty', 'diag_1',\n       'diag_2', 'diag_3', 'metformin', 'repaglinide',\n       'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',\n       'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',\n       'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',\n       'examide', 'citoglipton', 'insulin', 'glyburide-metformin',\n       'glipizide-metformin', 'glimepiride-pioglitazone',\n       'metformin-rosiglitazone', 'metformin-pioglitazone','change','diabetesMed'\n       ]\n"

In [18]:

#new_feature_df = pd.get_dummies(diabetes_data,prefix=categorical_columns, columns = categorical_columns , drop_first=True)

In [19]:
# split the dataset into train and test using stratified sampling to account for label imbalance
X_train, X_test, y_train, y_test = train_test_split(diabetes_data.loc[:,diabetes_data.columns!='label'], diabetes_data['label'],
                                                    stratify= diabetes_data['label'], 
                                                    test_size=0.25)

In [109]:
diabetes_data.to_csv('../data/processed/diabetes_data_processed.csv',sep='|',index=False)
X_train.to_csv('../data/processed/X_train.csv',sep='|',index=False)
y_train.to_csv('../data/processed/y_train.csv',sep='|',index=False)
X_test.to_csv('../data/processed/X_test.csv',sep='|',index=False)
y_test.to_csv('../data/processed/y_test.csv',sep='|',index=False)